# 🎯 IEEE-CIS Fraud Detection: Baseline модели

## Цель
Построить **baseline модели** с дефолтными параметрами для оценки качества детекции мошеннических транзакций. Это отправная точка перед оптимизацией.

## Датасет
- **Источник**: [IEEE-CIS Fraud Detection (Kaggle)](https://www.kaggle.com/c/ieee-fraud-detection)
- **Размер**: 590,540 транзакций, 434 признака
- **Целевая переменная**: `isFraud` (мошенничество = 1, легитимная транзакция = 0)
- **Дисбаланс классов**: ~3.5% fraud (сильный дисбаланс)
- **Временной характер**: данные отсортированы по `TransactionDT`

## Задача
Бинарная классификация с **временной структурой** → используем TimeSeriesSplit для валидации.

## Подход
Сравнение 4 алгоритмов:
1. **Logistic Regression** — простая линейная модель
2. **Random Forest** — классический ансамбль деревьев
3. **LightGBM** — быстрый gradient boosting
4. **CatBoost** — автоматическая обработка категорий

## Метрика
**ROC-AUC** — основная метрика соревнования (устойчива к дисбалансу классов)



🎯 BASELINE = модель с параметрами ПО УМОЛЧАНИЮ

Зачем нужен baseline:
1. Быстрая оценка качества данных (можно ли вообще предсказывать?)
2. Точка отсчета для сравнения улучшений
3. Проверка, что предобработка работает корректно
4. Понимание, какая модель лучше подходит для задачи

Что мы НЕ делаем в baseline:
- ❌ Не настраиваем гиперпараметры (C, max_depth, learning_rate...)
- ❌ Не используем feature engineering (новые признаки)
- ❌ Не делаем feature selection (отбор признаков)
- ❌ Не используем ансамбли
- ❌ НЕ используем early stopping (это тоже оптимизация!)

Что мы делаем в baseline:
- ✅ Минимальная предобработка (заполнение пропусков, encoding)
- ✅ Параметры по умолчанию
- ✅ Честная валидация (TimeSeriesSplit)
- ✅ Сравнение разных типов моделей





In [1]:
import os
import gc
from typing import List, Dict, Tuple
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# Настройки визуализации (работает везде)
sns.set_style("darkgrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Настройки pandas
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 120)
pd.set_option('display.max_colwidth', 50)


import pandas as pd
import numpy as np
from helper import *
from plots import *

print('✓ Окружение готово!')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, 
    recall_score, f1_score, classification_report, confusion_matrix
)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

✓ Окружение готово!


In [2]:
# ============================================================================
# 1. ЗАГРУЗКА И ПОДГОТОВКА
# ============================================================================

print("📂 Загрузка данных...")
train = load_data('data/train_transaction.csv', 'data/train_identity.csv')
train = train.sort_values('TransactionDT').reset_index(drop=True)

print(f"Train shape: {train.shape}")
print(f"Fraud rate: {train['isFraud'].mean():.2%}")

X = train.drop(['isFraud', 'TransactionID'], axis=1)
y = train['isFraud']

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\n📊 Числовые: {len(numeric_features)}, Категориальные: {len(categorical_features)}")

# ============================================================================
# 2. ПРЕДОБРАБОТКА ОДИН РАЗ (для простоты и скорости)
# ============================================================================

# Для Logistic Regression
X_logreg = X.copy()
imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()
X_logreg[numeric_features] = scaler.fit_transform(imputer.fit_transform(X_logreg[numeric_features]))
for col in categorical_features:
    le = LabelEncoder()
    X_logreg[col] = le.fit_transform(X_logreg[col].fillna('missing').astype(str))

# Для Random Forest
X_rf = X.copy()
X_rf[numeric_features] = imputer.fit_transform(X_rf[numeric_features])
for col in categorical_features:
    le = LabelEncoder()
    X_rf[col] = le.fit_transform(X_rf[col].fillna('missing').astype(str))

# Для LightGBM
X_lgb = X.copy()
for col in categorical_features:
    le = LabelEncoder()
    X_lgb[col] = le.fit_transform(X_lgb[col].fillna('missing').astype(str))

# Для CatBoost
X_cat = X.copy()
for col in categorical_features:
    X_cat[col] = X_cat[col].astype(str)

print("✅ Предобработка выполнена")

# ============================================================================
# 3. CROSS-VALIDATION
# ============================================================================

tscv = TimeSeriesSplit(n_splits=5)
splits = list(tscv.split(X))

models = {
    '1. Logistic Regression': (
        X_logreg, 
        LogisticRegression(
            max_iter=100,      # ← УСКОРЕНИЕ (default=100, оставляем явно)
            random_state=42, 
            n_jobs=-1
        )
    ),
    '2. Random Forest': (
        X_rf, 
        RandomForestClassifier(
            n_estimators=100,  # ← УСКОРЕНИЕ (default=100, оставляем явно)
            max_depth=10,      # ← УСКОРЕНИЕ (ограничение глубины)
            random_state=42, 
            n_jobs=-1
        )
    ),
    '3. LightGBM': (
        X_lgb, 
        LGBMClassifier(
            n_estimators=100,  # ← УСКОРЕНИЕ (default=100, оставляем явно)
            random_state=42, 
            n_jobs=-1, 
            verbose=-1
        )
    ),
    '4. CatBoost': (
        X_cat, 
        CatBoostClassifier(
            iterations=100,    # ← УСКОРЕНИЕ! (default=1000 → 100)
            cat_features=categorical_features, 
            random_state=42, 
            verbose=0, 
            thread_count=-1
        )
    )
}

print("\n" + "="*100)
print("✅ BASELINE МОДЕЛИ СОЗДАНЫ (ПАРАМЕТРЫ ПО УМОЛЧАНИЮ, БЕЗ EARLY STOPPING):")
print("="*100)
print("""
1. Logistic Regression:
   ✓ Label Encoding (категории → числа)
   ✓ Median Imputation (числовые пропуски)
   ✓ StandardScaler (масштабирование)
   → LogisticRegression(random_state=42) - DEFAULT!

2. Random Forest:
   ✓ Label Encoding (категории → числа)
   ✓ Median Imputation (числовые пропуски)
   → RandomForestClassifier(random_state=42) - DEFAULT!

3. LightGBM:
   ✓ Label Encoding (категории → числа)
   → LGBMClassifier(random_state=42) - DEFAULT!

4. CatBoost:
   ✓ Только приведение категорий к типу str
   → CatBoostClassifier(cat_features=..., random_state=42) - DEFAULT!
""")

print("\n" + "="*100)
print("🚀 BASELINE МОДЕЛИ (default параметры)")
print("="*100)

results = {}

for model_name, (X_processed, model) in models.items():
    print(f"\n{'='*100}\n🚀 {model_name}\n{'='*100}")
    
    cv_metrics = {'auc': [], 'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    
    for fold, (train_idx, val_idx) in enumerate(splits):
        X_tr, X_val = X_processed.iloc[train_idx], X_processed.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_tr, y_tr)
        
        y_pred_proba = model.predict_proba(X_val)[:,1]
        y_pred = model.predict(X_val)
        
        auc = roc_auc_score(y_val, y_pred_proba)
        cv_metrics['auc'].append(auc)
        cv_metrics['accuracy'].append(accuracy_score(y_val, y_pred))
        cv_metrics['precision'].append(precision_score(y_val, y_pred, zero_division=0))
        cv_metrics['recall'].append(recall_score(y_val, y_pred))
        cv_metrics['f1'].append(f1_score(y_val, y_pred))
        
        print(f"Fold {fold+1}: AUC={auc:.4f}")
    
    print(f"\n{'-'*100}\n📊 УСРЕДНЕННЫЕ МЕТРИКИ:\n{'-'*100}")
    for metric, values in cv_metrics.items():
        print(f"{metric.upper():12s} {np.mean(values):.4f} ± {np.std(values):.4f}")
    
    results[model_name] = {
        'mean_auc': np.mean(cv_metrics['auc']),
        'std_auc': np.std(cv_metrics['auc']),
        'mean_precision': np.mean(cv_metrics['precision']),
        'mean_recall': np.mean(cv_metrics['recall']),
        'mean_f1': np.mean(cv_metrics['f1'])
    }
    
    # Детальный анализ последнего фолда
    print(f"\n{'-'*100}\n📋 ПОСЛЕДНИЙ ФОЛД:\n{'-'*100}")
    
    train_idx_last, val_idx_last = splits[-1]
    model.fit(X_processed.iloc[train_idx_last], y.iloc[train_idx_last])
    y_pred_last = model.predict(X_processed.iloc[val_idx_last])
    
    print("\nClassification Report:")
    print(classification_report(y.iloc[val_idx_last], y_pred_last, target_names=['Legitimate', 'Fraud']))
    
    cm = confusion_matrix(y.iloc[val_idx_last], y_pred_last)
    print(f"\nConfusion Matrix:")
    print(f"                 Predicted")
    print(f"                 Legit  Fraud")
    print(f"Actual Legit    {cm[0,0]:6d} {cm[0,1]:6d}")
    print(f"       Fraud    {cm[1,0]:6d} {cm[1,1]:6d}")
    
    # Feature importance
    if hasattr(model, 'feature_importances_'):
        # Для Random Forest, LightGBM, CatBoost
        print(f"\n{'-'*100}\n🔍 ТОП-20 ПРИЗНАКОВ:\n{'-'*100}")
        importance = pd.DataFrame({
            'feature': X.columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False).head(20)
        
        for idx, row in importance.iterrows():
            print(f"{idx+1:2d}. {row['feature']:30s} {row['importance']:.6f}")
    
    elif hasattr(model, 'coef_'):
        # Для Logistic Regression
        print(f"\n{'-'*100}\n🔍 ТОП-20 ПРИЗНАКОВ (по абсолютному значению коэффициентов):\n{'-'*100}")
        
        coefficients = model.coef_[0]
        importance = pd.DataFrame({
            'feature': X.columns,
            'coefficient': coefficients,
            'abs_coefficient': np.abs(coefficients)
        }).sort_values('abs_coefficient', ascending=False).head(20)
        
        for idx, row in enumerate(importance.itertuples(), 1):
            direction = "↑ Fraud" if row.coefficient > 0 else "↓ Fraud"
            print(f"{idx:2d}. {row.feature:30s} {row.coefficient:+.4f}  ({direction})")

# ============================================================================
# 4. ИТОГОВОЕ СРАВНЕНИЕ
# ============================================================================

print(f"\n{'='*100}\n🏆 ИТОГОВОЕ СРАВНЕНИЕ\n{'='*100}")

comparison = pd.DataFrame({
    'Model': list(results.keys()),
    'Mean AUC': [results[m]['mean_auc'] for m in results.keys()],
    'Std AUC': [results[m]['std_auc'] for m in results.keys()],
    'Mean Precision': [results[m]['mean_precision'] for m in results.keys()],
    'Mean Recall': [results[m]['mean_recall'] for m in results.keys()],
    'Mean F1': [results[m]['mean_f1'] for m in results.keys()]
}).sort_values('Mean AUC', ascending=False)

print(f"\n{comparison.to_string(index=False)}")

print(f"\n{'='*100}")
print(f"🥇 ЛУЧШАЯ: {comparison.iloc[0]['Model']}")
print(f"   AUC: {comparison.iloc[0]['Mean AUC']:.4f} ± {comparison.iloc[0]['Std AUC']:.4f}")
print(f"{'='*100}")

comparison.to_csv('baseline_comparison.csv', index=False)
print("\n💾 Результаты: baseline_comparison.csv")
print("\n✅ ГОТОВО!")

📂 Загрузка данных...
Загрузка данных...
Датасет загружен: (590540, 434)
Train shape: (590540, 434)
Fraud rate: 3.50%

📊 Числовые: 401, Категориальные: 31
✅ Предобработка выполнена

✅ BASELINE МОДЕЛИ СОЗДАНЫ (ПАРАМЕТРЫ ПО УМОЛЧАНИЮ, БЕЗ EARLY STOPPING):

1. Logistic Regression:
   ✓ Label Encoding (категории → числа)
   ✓ Median Imputation (числовые пропуски)
   ✓ StandardScaler (масштабирование)
   → LogisticRegression(random_state=42) - DEFAULT!

2. Random Forest:
   ✓ Label Encoding (категории → числа)
   ✓ Median Imputation (числовые пропуски)
   → RandomForestClassifier(random_state=42) - DEFAULT!

3. LightGBM:
   ✓ Label Encoding (категории → числа)
   → LGBMClassifier(random_state=42) - DEFAULT!

4. CatBoost:
   ✓ Только приведение категорий к типу str
   → CatBoostClassifier(cat_features=..., random_state=42) - DEFAULT!


🚀 BASELINE МОДЕЛИ (default параметры)

🚀 1. Logistic Regression
Fold 1: AUC=0.7736
Fold 2: AUC=0.7920
Fold 3: AUC=0.7932
Fold 4: AUC=0.8055
Fold 5: AUC=0.7947


Fold 1: AUC=0.8562
Fold 2: AUC=0.8821
Fold 3: AUC=0.8644
Fold 4: AUC=0.8756
Fold 5: AUC=0.8816

----------------------------------------------------------------------------------------------------
📊 УСРЕДНЕННЫЕ МЕТРИКИ:
----------------------------------------------------------------------------------------------------
AUC          0.8720 ± 0.0102
ACCURACY     0.9722 ± 0.0017
PRECISION    0.7653 ± 0.0496
RECALL       0.3567 ± 0.0424
F1           0.4838 ± 0.0347

----------------------------------------------------------------------------------------------------
📋 ПОСЛЕДНИЙ ФОЛД:
----------------------------------------------------------------------------------------------------

Classification Report:
              precision    recall  f1-score   support

  Legitimate       0.98      0.99      0.99     95026
       Fraud       0.70      0.35      0.47      3397

    accuracy                           0.97     98423
   macro avg       0.84      0.67      0.73     98423
weighted avg     


 ### 🎯 КЛЮЧЕВЫЕ МОМЕНТЫ:

- 1. CatBoost — ЕДИНСТВЕННАЯ модель, которая:
   ✓ Работает с категориями как со строками (без encoding)
   ✓ Обрабатывает NaN в категориях и числах автоматически
   ✓ Использует Ordered Target Encoding (оптимально для fraud detection)

- 2. Почему CatBoost обрабатывает пропуски лучше:
   • В числовых: NaN трактуется как отдельное значение (может быть сигналом!)
   • В категориях: NaN может означать "отсутствие данных" (тоже паттерн)
   • Модель сама решает, как использовать эту информацию

- 3. Зачем CatBoostPreprocessor:
   • Только приводит категории к типу str (технический момент)
   • НЕ заполняет пропуски, НЕ кодирует категории
   • Минимальное вмешательство = максимум для CatBoost

⚡ ПРАКТИЧЕСКИЙ СОВЕТ:

Если у вас в данных:
• Много категорий (>5 признаков)
• Высокая кардинальность (email, device_id)
• Много пропусков (>10%)

→ CatBoost с cat_features даст лучший результат, чем любая другая модель!

Для IEEE-CIS это идеальный выбор: ProductCD, card*, addr*, P_emaildomain, etc.